# Loop Workflow Example 5

* This example is a how to for constructing your own thickness calculator
* It builds on Example 3 and using the local data in the source_data directory

In [ ]:
#if not already installed:
#!conda install -c loop3d map2loop loopstructural pyamg meshio

### Creating the replacement thickness calculator

In [ ]:
# Create a class that inherits from map2loop.sorter
from map2loop.thickness_calculator import ThicknessCalculator
from map2loop.mapdata import MapData
import geopandas
import pandas

class ThicknessCalculatorFixed(ThicknessCalculator):
    # Name this sorter in the init function
    # Add a variable for the fixed thickness value
    def __init__(self, thickness=100):
        self.sorter_label = "ThicknessCalculatorFixed"
        self.default_thickness = thickness

    # The structure of the sort function MUST match the Sorter sort function as it is
    # replacing that abstract function
    def compute(
        self,
        units: pandas.DataFrame,
        stratigraphic_order: list,
        basal_contacts: pandas.DataFrame,
        map_data: MapData,
    ) -> pandas.DataFrame:
        # The output of this function is the units pandas.dataframe
        # in this example we use the fixed value given at init
        output_units = units.copy()
        output_units['thickness'] = self.default_thickness
        return output_units

## Map2Loop (same as Example 3)

In [ ]:
import os
from map2loop.project import Project
from map2loop.m2l_enums import VerboseLevel
from map2loop.m2l_enums import Datatype
from map2loop.sampler import SamplerSpacing, SamplerDecimator
from map2loop.sorter import SorterUseHint, SorterUseNetworkX, SorterAgeBased, SorterAlpha
import time

from datetime import datetime
nowtime=datetime.now().isoformat(timespec='minutes')   
model_name=nowtime.replace(":","-").replace("T","-")
loop_project_filename = os.path.join(model_name, "local_source.loop3d")

# Specify the boundary of the region of interest in the appropriate projection coordinates
bounding_box = {
    "minx": 520000,
    "miny": 7490000,
    "maxx": 550000,
    "maxy": 7510000,
    "base": -3200,
    "top": 1200,
}
# Initialise the project with the shapefiles, dtm, config file
# output locations and projection to work in
proj = Project( 
    geology_filename = "./source_data/geol_clip.shp",
    fault_filename = "./source_data/faults_clip.shp",
    structure_filename = "./source_data/structure_clip.shp",
    dtm_filename = './source_data/dtm_rp.tif',
    config_filename = './source_data/example.hjson',
    clut_filename = './source_data/500kibg_colours.csv',
    clut_file_legacy = True,
    verbose_level = VerboseLevel.NONE,
    tmp_path = model_name,
    working_projection = "EPSG:28350",
    bounding_box = bounding_box,
    loop_project_filename = loop_project_filename
)

# Remove faults less than 5km
proj.set_minimum_fault_length(5000.0)

# Set sampling distance for geology and fault maps to 200m
proj.set_sampler(Datatype.GEOLOGY, SamplerSpacing(200.0))
proj.set_sampler(Datatype.FAULT, SamplerSpacing(200.0))

# Set to only take every second orientation observation (0 or 1 means take all observations)
proj.set_sampler(Datatype.STRUCTURE, SamplerDecimator(2))

# Set what text is expected for intrusions (contained within the description field)
proj.map_data.config.geology_config["intrusive_text"] = "mafic intrusive"

# Set specific layers from the geology map to be ignored (commonly "cover" or "water")
proj.set_ignore_codes(["cover", "Fortescue_Group", "A_FO_od"])

### Inserting the thickness calculator into the map2loop process 

In [ ]:
# This step sets the newly created sorter as the one map2loop will use
proj.set_thickness_calculator(ThicknessCalculatorFixed(50))
proj.run_all()

### Checking the units thickness values

In [ ]:
print(proj.stratigraphic_column.stratigraphicUnits[['name','thickness']])